## A ANN for classification

In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.cluster import KMeans

In [2]:
# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Linear(input_dim, encoding_dim)
        self.decoder = nn.Linear(encoding_dim, input_dim)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [3]:
# generate an array of 20 with random integer values between 0 and 100
x = np.random.randint(0, 100, 20)

In [4]:
x

array([14, 53, 12, 92, 14, 64, 16, 31, 62, 14, 70, 60, 39, 79,  9, 19, 86,
       25, 86, 16])

In [5]:
# Convert the array to a PyTorch tensor
tensor_values = torch.tensor(x, dtype=torch.float32).view(-1, 1)

In [6]:
# Define the dimensions of the autoencoder
input_dim = 1
encoding_dim = 2

In [7]:
# Train the autoencoder
autoencoder = Autoencoder(input_dim, encoding_dim)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.001, weight_decay=0.0001)

In [8]:
num_epochs = 100
for epoch in range(num_epochs):
    encoded, decoded = autoencoder(tensor_values)
    loss = criterion(decoded, tensor_values)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [10]:
# Get the encoded representations for all values
encoded_values, _ = autoencoder(tensor_values)

In [11]:
encoded_values

tensor([[ -1.3790,  -6.6266],
        [ -6.6824, -26.0086],
        [ -1.1071,  -5.6326],
        [-11.9858, -45.3906],
        [ -1.3790,  -6.6266],
        [ -8.1782, -31.4753],
        [ -1.6510,  -7.6205],
        [ -3.6908, -15.0752],
        [ -7.9063, -30.4814],
        [ -1.3790,  -6.6266],
        [ -8.9942, -34.4572],
        [ -7.6343, -29.4874],
        [ -4.7786, -19.0510],
        [-10.2180, -38.9300],
        [ -0.6991,  -4.1417],
        [ -2.0590,  -9.1114],
        [-11.1699, -42.4088],
        [ -2.8749, -12.0933],
        [-11.1699, -42.4088],
        [ -1.6510,  -7.6205]], grad_fn=<AddmmBackward0>)

In [12]:
# Determine the optimal number of groups using the elbow method
distortions = []
max_clusters = len(y)
for k in range(1, max_clusters + 1):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(encoded_values.detach().numpy())
    distortions.append(kmeans.inertia_)

c:\Users\erik_\miniconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\erik_\miniconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\erik_\miniconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\erik_\miniconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the envir

In [13]:
# Find the optimal number of clusters using the elbow point in the distortion plot
elbow_point = 0
for i in range(1, max_clusters - 1):
    prev_slope = distortions[i] - distortions[i - 1]
    next_slope = distortions[i + 1] - distortions[i]
    if prev_slope > next_slope:
        elbow_point = i + 1
        break

In [14]:
# Group the values using K-means with the optimal number of clusters
kmeans = KMeans(n_clusters=elbow_point)
kmeans.fit(encoded_values.detach().numpy())
cluster_labels = kmeans.labels_

c:\Users\erik_\miniconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\erik_\miniconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [15]:
elbow_point

8

In [16]:
# Print the grouped values
for i in range(elbow_point):
    cluster = [x[j] for j, label in enumerate(cluster_labels) if label == i]
    print(f"Cluster {i + 1}: {cluster}")

Cluster 1: [14, 12, 14, 16, 14, 9, 19, 16]
Cluster 2: [70]
Cluster 3: [92, 86, 86]
Cluster 4: [39]
Cluster 5: [53]
Cluster 6: [31, 25]
Cluster 7: [64, 62, 60]
Cluster 8: [79]
